### Download and Store Data Files

In [1]:
import os
import ssl
import sys
import urllib.request
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
pd.set_option('display.max_columns', None)

#### Downloading the Files

In [2]:
gradurls = {'https://infohub.nyced.org/docs/default-source/default-document-library/2020-graduation_rates_public_school.xlsx': 'grad_rates.xlsx',
            'https://infohub.nyced.org/docs/default-source/default-document-library/2020-graduation_rates_public_charters.xlsx': 'charter_grad_rates.xlsx'}
sqrurls = {'https://infohub.nyced.org/docs/default-source/default-document-library/201920_hs_sqr_results.xlsx': 'sqr2020.xlsx',
           'https://infohub.nyced.org/docs/default-source/default-document-library/201819_hs_sqr_results.xlsx': 'sqr2019.xlsx',
           'https://infohub.nyced.org/docs/default-source/default-document-library/201718_hs_sqr_results.xlsx': 'sqr2018.xlsx',
           'http://infohub.nyced.org/docs/default-source/default-document-library/2016-17_hs_sqr.xlsx': 'sqr2017.xlsx',
           'http://infohub.nyced.org/docs/default-source/default-document-library/2015_2016_hs_sqr_results_2017_01_05.xlsx': 'sqr2016.xlsx',
           'http://infohub.nyced.org/docs/default-source/default-document-library/2014_2015_hs_sqr_results_2016_04_08.xlsx': 'sqr2015.xlsx'}
demourls = {'https://infohub.nyced.org/docs/default-source/default-document-library/demographic-snapshot-2015-16-to-2019-20-(public).xlsx': 'demo.xlsx'}
urls = [gradurls, sqrurls, demourls]

In [3]:
#import ssl
#ssl._create_default_https_context = ssl._create_unverified_context
for url in urls:
    try:
        [urllib.request.urlretrieve(x, 'data/'+url[x]) for x in url]
    except:
        print("Trying the next url")

Trying the next url
Trying the next url
Trying the next url


#### Graduation Rate Files

In [4]:
gradrates = pd.read_excel('data/grad_rates.xlsx', sheet_name = "All")
charterrates = pd.read_excel('data/charter_grad_rates.xlsx', sheet_name = "All")

gradrates_eth = pd.read_excel('data/grad_rates.xlsx', sheet_name = "Ethnicity")

In [5]:
gradrates[gradrates['Cohort']=='4 year June'].groupby(['Cohort Year']).count()['DBN']

Cohort Year
2001    262
2002    318
2003    346
2004    368
2005    389
2006    406
2007    419
2008    436
2009    449
2010    473
2011    467
2012    474
2013    477
2014    479
2015    480
2016    475
Name: DBN, dtype: int64

In [6]:
gradrates_eth.head()

,Unnamed: 0,DBN,School Name,Category,Cohort Year,Cohort,# Total Cohort,# Grads,% Grads,# Total Regents,% Total Regents of Cohort,% Total Regents of Grads,# Advanced Regents,% Advanced Regents of Cohort,% Advanced Regents of Grads,# Regents without Advanced,% Regents without Advanced of Cohort,% Regents without Advanced of Grads,# Local,% Local of Cohort,% Local of Grads,# Still Enrolled,% Still Enrolled,# Dropout,% Dropout
0,01M2922016Asian4 year August,01M292,ORCHARD COLLEGIATE ACADEMY,Asian,2016,4 year August,7,6,85.714287,6,85.714287,100,3,42.857143,50,3,42.857143,50,0,0,0,0,0,1,14.285714
1,01M2922015Asian4 year August,01M292,ORCHARD COLLEGIATE ACADEMY,Asian,2015,4 year August,3,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s
2,01M2922014Asian4 year August,01M292,ORCHARD COLLEGIATE ACADEMY,Asian,2014,4 year August,3,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s
3,01M2922013Asian4 year August,01M292,ORCHARD COLLEGIATE ACADEMY,Asian,2013,4 year August,9,8,88.888886,8,88.888886,100,0,0,0,8,88.888886,100,0,0,0,0,0,1,11.111111
4,01M2922012Asian4 year August,01M292,ORCHARD COLLEGIATE ACADEMY,Asian,2012,4 year August,5,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s


In [7]:
charterrates.groupby('Cohort').count()

,DBN,School Name,Category,Cohort Year,# Total Cohort,# Grads,% Grads,# Total Regents,% Total Regents of Cohort,% Total Regents of Grads,# Advanced Regents,% Advanced Regents of Cohort,% Advanced Regents of Grads,# Regents without Advanced,% Regents without Advanced of Cohort,% Regents without Advanced of Grads,# Local,% Local of Cohort,% Local of Grads,# Still Enrolled,% Still Enrolled,# Dropout,% Dropout
Cohort,,,,,,,,,,,,,,,,,,,,,,,
4 year August,409,409,409,409,409,409,409,409,409,397,409,409,397,409,409,397,409,409,397,409,409,409,409
4 year June,409,409,409,409,409,409,409,409,409,389,409,409,389,409,409,389,409,409,389,409,409,409,409
5 year August,358,358,358,358,358,358,358,358,358,355,358,358,355,358,358,355,358,358,355,358,358,358,358
5 year June,358,358,358,358,358,358,358,358,358,354,358,358,354,358,358,354,358,358,354,358,358,358,358
6 year June,308,308,308,308,308,308,308,308,308,304,308,308,304,308,308,304,308,308,304,308,308,308,308


In [8]:
charterrates[charterrates['Cohort']=='4 year June'].groupby(['Cohort Year']).count()['DBN']

Cohort Year
2007     9
2008    18
2009    29
2010    35
2011    38
2012    47
2013    50
2014    57
2015    61
2016    65
Name: DBN, dtype: int64

In [9]:
all_grad_rates = pd.concat([gradrates[['DBN', 'School Name', 'Cohort Year', 'Cohort', '# Total Cohort', '# Grads', '% Grads']],
                            charterrates[['DBN', 'School Name', 'Cohort Year', 'Cohort', '# Total Cohort', '# Grads', '% Grads']]], 
                           ignore_index=True)
all_grad_rates[['# Total Cohort', '# Grads', '% Grads']] = all_grad_rates[['# Total Cohort', '# Grads', '% Grads']].apply(pd.to_numeric, errors='coerce')

In [10]:
all_grad_rates.rename(columns={'DBN':'dbn',
                               'School Name':'schoolName',
                               'Cohort Year':'cohortYear', 
                               'Cohort':'cohort', 
                               '# Total Cohort':'numTotalCohort',
                               '# Grads':'numGrads', '% Grads':'pctGrad', '# Still Enrolled':'stillEnrolledNum'}, inplace=True)

In [11]:
all_grad_rates['pctGrad'] = all_grad_rates['pctGrad']/100 

In [12]:
all_grad_rates.to_csv('data/clean/all_grad_rates.csv',index=False)

In [13]:
all_grad_rates.head()

,dbn,schoolName,cohortYear,cohort,numTotalCohort,numGrads,pctGrad
0,01M292,ORCHARD COLLEGIATE ACADEMY,2016,4 year August,31,29.0,0.935484
1,01M292,ORCHARD COLLEGIATE ACADEMY,2015,4 year August,28,25.0,0.892857
2,01M292,ORCHARD COLLEGIATE ACADEMY,2014,4 year August,29,28.0,0.965517
3,01M292,ORCHARD COLLEGIATE ACADEMY,2013,4 year August,36,25.0,0.694444
4,01M292,ORCHARD COLLEGIATE ACADEMY,2012,4 year August,44,24.0,0.545455


In [14]:
all_grad_rates[(all_grad_rates['dbn']=='12X267') & (all_grad_rates['cohort']=='4 year August')]

,dbn,schoolName,cohortYear,cohort,numTotalCohort,numGrads,pctGrad
14167,12X267,BRONX LATIN,2016,4 year August,83,78.0,0.939759
14168,12X267,BRONX LATIN,2015,4 year August,91,82.0,0.901099
14169,12X267,BRONX LATIN,2014,4 year August,81,76.0,0.938272
14170,12X267,BRONX LATIN,2013,4 year August,71,68.0,0.957747
14171,12X267,BRONX LATIN,2012,4 year August,62,56.0,0.903226
14172,12X267,BRONX LATIN,2011,4 year August,58,49.0,0.844828
14173,12X267,BRONX LATIN,2010,4 year August,56,50.0,0.892857
14174,12X267,BRONX LATIN,2009,4 year August,40,29.0,0.725000
14175,12X267,BRONX LATIN,2008,4 year August,50,38.0,0.760000
14176,12X267,BRONX LATIN,2007,4 year August,40,31.0,0.775000


#### SQR Files

In [15]:
tabs = ['Summary', 'Student Achievement', 'Closing the Achievement Gap', 'Framework', 'Additional Info']
sqr_files = ['sqr2020.xlsx', 'sqr2019.xlsx', 'sqr2018.xlsx', 'sqr2017.xlsx', 'sqr2016.xlsx', 'sqr2015.xlsx']

In [16]:
summary = {}
stud_achieve = {}
closing_gap = {}
framework = {}
add_info = {}

for f in sqr_files:
    summary[f] = pd.read_excel('data/'+f, sheet_name='Summary')
    stud_achieve[f] = pd.read_excel('data/'+f, sheet_name='Student Achievement')
    closing_gap[f] = pd.read_excel('data/'+f, sheet_name='Closing the Achievement Gap')
    framework[f] = pd.read_excel('data/'+f, sheet_name='Framework')
    add_info[f] = pd.read_excel('data/'+f, sheet_name='Additional Info')

In [17]:
summary['sqr2020.xlsx'].head()

,Unnamed: 0,Summary,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,"Due to COVID-19, Framework ratings and scores ...",NaN,DBN,School Name,School Type,Enrollment,Rigorous Instruction - Percent Positive,Collaborative Teachers - Percent Positive,Supportive Environment - Percent Positive,Effective School Leadership - Percent Positive,Strong Family-Community Ties - Percent Positive,Trust - Percent Positive,Quality Review - How interesting and challengi...,Quality Review - How effective is the teaching...,Quality Review - How well does the school asse...,Quality Review - How clearly are high expectat...,Quality Review - How well do teachers work wit...,Quality Review - How safe and inclusive is the...,Quality Review - How well does the school allo...,Quality Review - How well does the school iden...,Quality Review - How thoughtful is the school'...,Quality Review - How well are school decisions...,Quality Review - Dates of Review,Average Grade 8 English Proficiency,Average Grade 8 Math Proficiency,Percent Female,Percent Male,Percent English Language Learners,Percent Students with Disabilities,Percent Self-Contained,Economic Need Index,Percent Overage / Undercredited,Percent in Temp Housing,Percent HRA Eligible,Student Percent - Asian,Student Percent - Black,Student Percent - Hispanic,Student Percent - Native American,Student Percent - Native Hawaiian or Pacific I...,Student Percent - White,Teacher Percent - Asian,Teacher Percent - Black,Teacher Percent - Hispanic,Teacher Percent - Native American,Teacher Percent - Native Hawaiian or Pacific I...,Teacher Percent - White,Nearby Student Distance (mi),Nearby Student Percent - Asian,Nearby Student Percent - Black,Nearby Student Percent - Hispanic,Nearby Student Percent - Native American,Nearby Student Percent - Native Hawaiian or Pa...,Nearby Student Percent - White,Borough Percent - Asian,Borough Percent - Black,Borough Percent - Hispanic,Borough Percent - Native American,Borough Percent - Native Hawaiian or Pacific I...,Borough Percent - White,Percentage of Students Enrolled in Advanced Co...,Percentage of Students Enrolled in Advanced Co...,Percentage of Students Enrolled in Advanced Co...,Percentage of Students Enrolled in Advanced Co...,Percentage of Students Enrolled in Advanced Co...,Percentage of Students Enrolled in Advanced Co...,Percentage of Students Enrolled in AP or IB Co...,Percentage of Students Enrolled in AP or IB Co...,Percentage of Students Enrolled in AP or IB Co...,Percentage of Students Enrolled in AP or IB Co...,Percentage of Students Enrolled in AP or IB Co...,Percentage of Students Enrolled in AP or IB Co...,Years of principal experience at this school,Percent of teachers with 3 or more years of ex...,Average Student Attendance (through Feb-2020 o...,Percent of Students Chronically Absent (throug...,Teacher Attendance Rate

In [18]:
summary['sqr2020.xlsx'].columns = summary['sqr2020.xlsx'].iloc[1]
summary['sqr2019.xlsx'].columns = summary['sqr2019.xlsx'].iloc[1]
summary['sqr2018.xlsx'].columns = summary['sqr2018.xlsx'].iloc[0]
summary['sqr2017.xlsx'].columns = summary['sqr2017.xlsx'].iloc[0]
summary['sqr2016.xlsx'].columns = summary['sqr2016.xlsx'].iloc[0]
summary['sqr2015.xlsx'].columns = summary['sqr2015.xlsx'].iloc[0]

In [19]:
summary['sqr2020.xlsx'].columns

Index([                                                                                                           nan,
       'Due to COVID-19, Framework ratings and scores are not available for any schools for the 2019-20 school year.',
                                                                                                                  nan,
                                                                                                                'DBN',
                                                                                                        'School Name',
                                                                                                        'School Type',
                                                                                                         'Enrollment',
                                                                            'Rigorous Instruction - Percent Positive',
                                                

In [20]:
summary_cols = ['DBN','Enrollment', 'Percent Asian', 'Percent Black','Percent Hispanic', 'Percent White',
                'Percent English Language Learners','Percent Students with Disabilities','Percent Self-Contained',
                'Economic Need Index','Average Grade 8 English Proficiency','Average Grade 8 Math Proficiency',
                'Student Attendance Rate', 'Percent of Students Chronically Absent']
summary['sqr2019.xlsx'] = summary['sqr2019.xlsx'][summary_cols]
summary['sqr2018.xlsx'] = summary['sqr2018.xlsx'][summary_cols]
summary['sqr2017.xlsx'] = summary['sqr2017.xlsx'][summary_cols]
summary['sqr2016.xlsx'] = summary['sqr2016.xlsx'][summary_cols]
summary['sqr2015.xlsx'] = summary['sqr2015.xlsx'][summary_cols]

In [21]:
summary['sqr2020.xlsx'] = summary['sqr2020.xlsx'][['DBN','Enrollment',
                         'Student Percent - Asian','Student Percent - Black','Student Percent - Hispanic',
                         'Student Percent - White', 'Percent English Language Learners','Percent Students with Disabilities','Percent Self-Contained',
                         'Economic Need Index','Average Grade 8 English Proficiency','Average Grade 8 Math Proficiency',
                         'Average Student Attendance (through Feb-2020 only)',
                         'Percent of Students Chronically Absent (through Feb-2020 only)']]

In [22]:
summary['sqr2020.xlsx'].rename(columns={'Student Percent - Asian':'Percent Asian',
                                        'Student Percent - Black':'Percent Black',
                                        'Student Percent - Hispanic': 'Percent Hispanic',
                                        'Student Percent - White': 'Percent White', 
                                        'Average Student Attendance (through Feb-2020 only)' : 'Student Attendance Rate',
                                        'Percent of Students Chronically Absent (through Feb-2020 only)':'Percent of Students Chronically Absent'}, inplace=True)

In [23]:
summary['sqr2020.xlsx']['SQR Year'] = '2020'
summary['sqr2019.xlsx']['SQR Year'] = '2019'
summary['sqr2018.xlsx']['SQR Year'] = '2018'
summary['sqr2017.xlsx']['SQR Year'] = '2017'
summary['sqr2016.xlsx']['SQR Year'] = '2016'
summary['sqr2015.xlsx']['SQR Year'] = '2015'

In [24]:
summary_df = pd.concat(summary).reset_index().drop(['level_0', 'level_1'], axis = 1)
summary_df = summary_df[summary_df.DBN != 'DBN']

In [25]:
summary_df[['Enrollment', 'Percent Asian', 'Percent Black','Percent Hispanic', 'Percent White',
                'Percent English Language Learners','Percent Students with Disabilities','Percent Self-Contained',
                'Economic Need Index','Average Grade 8 English Proficiency','Average Grade 8 Math Proficiency',
                'Student Attendance Rate', 'Percent of Students Chronically Absent']] = summary_df[['Enrollment', 'Percent Asian', 'Percent Black','Percent Hispanic', 'Percent White',
                'Percent English Language Learners','Percent Students with Disabilities','Percent Self-Contained',
                'Economic Need Index','Average Grade 8 English Proficiency','Average Grade 8 Math Proficiency',
                'Student Attendance Rate', 'Percent of Students Chronically Absent']].apply(pd.to_numeric, errors='coerce')

In [26]:
summary_df.drop(columns=['Percent Asian', 'Percent Black',
       'Percent Hispanic', 'Percent White',
       'Percent English Language Learners',
       'Percent Students with Disabilities', 'Percent Self-Contained'], inplace=True)

summary_df.rename(columns={"DBN":'dbn', 'Enrollment':'sqrEnrollment',
                       "Average Grade 8 English Proficiency": 'grd8Ela',
                       "Average Grade 8 Math Proficiency": 'grd8Math',
                       "Student Attendance Rate": 'attdRate',
                       "Percent of Students Chronically Absent" : 'pctChronAbs',
                           "Economic Need Index": 'econNeedInd',
                       "SQR Year": 'sqrYear'}, inplace=True)

In [27]:
stud_achieve['sqr2020.xlsx'].columns = stud_achieve['sqr2020.xlsx'].iloc[1]
stud_achieve['sqr2019.xlsx'].columns = stud_achieve['sqr2019.xlsx'].iloc[1]
stud_achieve['sqr2018.xlsx'].columns = stud_achieve['sqr2018.xlsx'].iloc[0]
stud_achieve['sqr2017.xlsx'].columns = stud_achieve['sqr2017.xlsx'].iloc[0]
stud_achieve['sqr2016.xlsx'].columns = stud_achieve['sqr2016.xlsx'].iloc[0]
stud_achieve['sqr2015.xlsx'].columns = stud_achieve['sqr2015.xlsx'].iloc[0]

In [28]:
stud_achieve['sqr2020.xlsx'].columns 

Index([                                                                                                           nan,
       'Due to COVID-19, Framework ratings and scores are not available for any schools for the 2019-20 school year.',
                                                                                                                  nan,
                                                                                                                'DBN',
                                                                                                        'School Name',
                                                                                                        'School Type',
                                                                   'N count - 10+ Credits in 1st Year - All Students',
                                                              'Metric Value - 10+ Credits in 1st Year - All Students',
                                                

In [29]:
sqrcols = ["DBN", "N count - 10+ Credits in 1st Year - All Students",
                              "Metric Value - 10+ Credits in 1st Year - All Students",
                              "N count - 10+ Credits in 1st Year - School's Lowest Third",
                              "Metric Value - 10+ Credits in 1st Year - School's Lowest Third",
                              "N count - 10+ Credits in 2nd Year - All Students",
                              "Metric Value - 10+ Credits in 2nd Year - All Students",
                              "N count - 10+ Credits in 2nd Year - School's Lowest Third",
                              "Metric Value - 10+ Credits in 2nd Year - School's Lowest Third",
                              "N count - 10+ Credits in 3rd Year - All Students",
                              "Metric Value - 10+ Credits in 3rd Year - All Students",
                              "N count - 10+ Credits in 3rd Year - School's Lowest Third",
                              "Metric Value - 10+ Credits in 3rd Year - School's Lowest Third"]
stud_achieve['sqr2020.xlsx'] = stud_achieve['sqr2020.xlsx'][sqrcols]
stud_achieve['sqr2019.xlsx'] = stud_achieve['sqr2019.xlsx'][sqrcols]
stud_achieve['sqr2018.xlsx'] = stud_achieve['sqr2018.xlsx'][sqrcols]
stud_achieve['sqr2017.xlsx'] = stud_achieve['sqr2017.xlsx'][sqrcols]

sqrcols = ["DBN", "Metric Value - Percentage Earning 10+ Credits in First Year",
           "Metric Value - Lowest Third School, Percentage Earning 10+ Credits in First Year",
           "Metric Value - Percentage Earning 10+ Credits in Second Year",
           "Metric Value - Lowest Third School, Percentage Earning 10+ Credits in Second Year",
           "Metric Value - Percentage Earning 10+ Credits in Third Year",
           "Metric Value - Lowest Third School, Percentage Earning 10+ Credits in Third Year"]
stud_achieve['sqr2015.xlsx'] = stud_achieve['sqr2015.xlsx'][sqrcols]
stud_achieve['sqr2016.xlsx'] = stud_achieve['sqr2016.xlsx'][sqrcols]

In [30]:
stud_achieve['sqr2015.xlsx'].columns

Index(['DBN', 'Metric Value - Percentage Earning 10+ Credits in First Year',
       'Metric Value - Lowest Third School, Percentage Earning 10+ Credits in First Year',
       'Metric Value - Percentage Earning 10+ Credits in Second Year',
       'Metric Value - Lowest Third School, Percentage Earning 10+ Credits in Second Year',
       'Metric Value - Percentage Earning 10+ Credits in Third Year',
       'Metric Value - Lowest Third School, Percentage Earning 10+ Credits in Third Year'],
      dtype='object', name=0)

In [31]:
sqrcols = {"Metric Value - Percentage Earning 10+ Credits in First Year":"Metric Value - 10+ Credits in 1st Year - All Students",
           "Metric Value - Lowest Third School, Percentage Earning 10+ Credits in First Year": "Metric Value - 10+ Credits in 1st Year - School's Lowest Third",
           "Metric Value - Percentage Earning 10+ Credits in Second Year" : "Metric Value - 10+ Credits in 2nd Year - All Students",
           "Metric Value - Lowest Third School, Percentage Earning 10+ Credits in Second Year": "Metric Value - 10+ Credits in 2nd Year - School's Lowest Third",
           "Metric Value - Percentage Earning 10+ Credits in Third Year": "Metric Value - 10+ Credits in 3rd Year - All Students",
           "Metric Value - Lowest Third School, Percentage Earning 10+ Credits in Third Year": "Metric Value - 10+ Credits in 3rd Year - School's Lowest Third"}
stud_achieve['sqr2015.xlsx'].rename(columns = sqrcols, inplace=True)
stud_achieve['sqr2016.xlsx'].rename(columns = sqrcols, inplace=True)

In [32]:
stud_achieve['sqr2020.xlsx']['SQR Year'] = '2020'
stud_achieve['sqr2019.xlsx']['SQR Year'] = '2019'
stud_achieve['sqr2018.xlsx']['SQR Year'] = '2018'
stud_achieve['sqr2017.xlsx']['SQR Year'] = '2017'
stud_achieve['sqr2016.xlsx']['SQR Year'] = '2016'
stud_achieve['sqr2015.xlsx']['SQR Year'] = '2015'

In [33]:
stud_achieve_df = pd.concat(stud_achieve).reset_index().drop(['level_0', 'level_1'], axis=1)
stud_achieve_df = stud_achieve_df[stud_achieve_df.DBN != 'DBN']

In [34]:
stud_achieve_df[["N count - 10+ Credits in 1st Year - All Students", "Metric Value - 10+ Credits in 1st Year - All Students",
                 "N count - 10+ Credits in 1st Year - School's Lowest Third",
                 "Metric Value - 10+ Credits in 1st Year - School's Lowest Third",
                 "N count - 10+ Credits in 2nd Year - All Students",
                 "Metric Value - 10+ Credits in 2nd Year - All Students",
                 "N count - 10+ Credits in 2nd Year - School's Lowest Third",
                 "Metric Value - 10+ Credits in 2nd Year - School's Lowest Third",
                 "N count - 10+ Credits in 3rd Year - All Students",
                 "Metric Value - 10+ Credits in 3rd Year - All Students",
                 "N count - 10+ Credits in 3rd Year - School's Lowest Third",
                 "Metric Value - 10+ Credits in 3rd Year - School's Lowest Third"]] = stud_achieve_df[["N count - 10+ Credits in 1st Year - All Students",
                              "Metric Value - 10+ Credits in 1st Year - All Students",
                              "N count - 10+ Credits in 1st Year - School's Lowest Third",
                              "Metric Value - 10+ Credits in 1st Year - School's Lowest Third",
                              "N count - 10+ Credits in 2nd Year - All Students",
                              "Metric Value - 10+ Credits in 2nd Year - All Students",
                              "N count - 10+ Credits in 2nd Year - School's Lowest Third",
                              "Metric Value - 10+ Credits in 2nd Year - School's Lowest Third",
                              "N count - 10+ Credits in 3rd Year - All Students",
                              "Metric Value - 10+ Credits in 3rd Year - All Students",
                              "N count - 10+ Credits in 3rd Year - School's Lowest Third",
                              "Metric Value - 10+ Credits in 3rd Year - School's Lowest Third"]].apply(pd.to_numeric, errors='coerce')

In [35]:
stud_achieve_df.rename(columns={"DBN":'dbn', 'Enrollment':'sqrEnrollment',
                       "Average Grade 8 English Proficiency": 'grd8Ela',
                       "Average Grade 8 Math Proficiency": 'grd8Math',
                       "Student Attendance Rate": 'attdRate',
                       "Percent of Students Chronically Absent" : 'pctChronAbs',
                       "SQR Year": 'sqrYear',
                       "Metric Value - 10+ Credits in 1st Year - All Students" : 'pct10PlusYear1',
                       "Metric Value - 10+ Credits in 1st Year - School's Lowest Third" : 'pct10PlusYear1Lowest3rd',
                       "Metric Value - 10+ Credits in 2nd Year - All Students" : 'pct10PlusYear2',
                       "Metric Value - 10+ Credits in 2nd Year - School's Lowest Third" : 'pct10PlusYear2Lowest3rd',
                       "Metric Value - 10+ Credits in 3rd Year - All Students" : 'pct10PlusYear3',
                       "Metric Value - 10+ Credits in 3rd Year - School's Lowest Third" : 'pct10PlusYear3Lowest3rd',
                       "N count - 10+ Credits in 1st Year - All Students" : 'numYear1',
                       "N count - 10+ Credits in 1st Year - School's Lowest Third" : 'numYear1Lowest3rd',
                       "N count - 10+ Credits in 2nd Year - All Students" : 'numYear2',
                       "N count - 10+ Credits in 2nd Year - School's Lowest Third" : 'numYear2Lowest3rd',
                       "N count - 10+ Credits in 3rd Year - All Students" : 'numYear3',
                       "N count - 10+ Credits in 3rd Year - School's Lowest Third": 'numYear3Lowest3rd'}, inplace=True)

In [36]:
closing_gap['sqr2020.xlsx'].columns = closing_gap['sqr2020.xlsx'].iloc[1]
closing_gap['sqr2019.xlsx'].columns = closing_gap['sqr2019.xlsx'].iloc[1]
closing_gap['sqr2018.xlsx'].columns = closing_gap['sqr2018.xlsx'].iloc[0]
closing_gap['sqr2017.xlsx'].columns = closing_gap['sqr2017.xlsx'].iloc[0]
closing_gap['sqr2016.xlsx'].columns = closing_gap['sqr2016.xlsx'].iloc[0]
closing_gap['sqr2015.xlsx'].columns = closing_gap['sqr2015.xlsx'].iloc[0]

closing_gap['sqr2020.xlsx']['sqrYear'] = '2020'
closing_gap['sqr2019.xlsx']['sqrYear'] = '2019'
closing_gap['sqr2018.xlsx']['sqrYear'] = '2018'
closing_gap['sqr2017.xlsx']['sqrYear'] = '2017'
closing_gap['sqr2016.xlsx']['sqrYear'] = '2016'
closing_gap['sqr2015.xlsx']['sqrYear'] = '2015'

In [37]:
closing_gap['sqr2020.xlsx'].columns

Index([                                                                                                           nan,
       'Due to COVID-19, Framework ratings and scores are not available for any schools for the 2019-20 school year.',
                                                                                                                  nan,
                                                                                                                'DBN',
                                                                                                        'School Name',
                                                                                                        'School Type',
                                                                           'N count - 4-Year Graduation Rate - Black',
                                                                      'Metric Value - 4-Year Graduation Rate - Black',
                                                

In [38]:
closing_gap['sqr2020.xlsx'] = closing_gap['sqr2020.xlsx'][['DBN', 'N count - 4-Year Graduation Rate - Lowest Third Citywide',
                             'Metric Value - 4-Year Graduation Rate - Lowest Third Citywide',
                             'N count - 4-Year College Readiness Index, Lowest Third Citywide (without CAT)',
                             'Metric Value - 4-Year College Readiness Index, Lowest Third Citywide (without CAT)',
                             'N count - Postsecondary Enrollment Rate - 6 Months, Lowest Third Citywide',
                             'Metric Value - Postsecondary Enrollment Rate - 6 Months, Lowest Third Citywide',
                             'N count - College and Career Preparatory Course Index, Lowest Third Citywide',
                             'Metric Value - College and Career Preparatory Course Index, Lowest Third Citywide', 'sqrYear']]
closing_gap['sqr2020.xlsx'].rename(columns={'DBN':'dbn',
                                            'N count - 4-Year Graduation Rate - Lowest Third Citywide': 'numGradLowest3rd',
                             'Metric Value - 4-Year Graduation Rate - Lowest Third Citywide': 'pctGradLowest3rd',
                             'N count - 4-Year College Readiness Index, Lowest Third Citywide (without CAT)': 'numCollegeReadyLowest3rd',
                             'Metric Value - 4-Year College Readiness Index, Lowest Third Citywide (without CAT)': 'pctCollegeReadyLowest3rd',
                             'N count - Postsecondary Enrollment Rate - 6 Months, Lowest Third Citywide': 'numPERLowest3rd' ,
                             'Metric Value - Postsecondary Enrollment Rate - 6 Months, Lowest Third Citywide': 'pctPERLowest3rd',
                             'N count - College and Career Preparatory Course Index, Lowest Third Citywide': 'numCCPCILowest3rd',
                             'Metric Value - College and Career Preparatory Course Index, Lowest Third Citywide': 'pctCCPCILowest3rd'}, inplace=True)

In [39]:
closing_gap_cols = ['DBN', 'N count - 4-Year Graduation Rate - Lowest Third Citywide',
                    'Metric Value - 4-Year Graduation Rate - Lowest Third Citywide',
                    'N count - 4-Year College Readiness Index, Lowest Third Citywide',
                    'Metric Value - 4-Year College Readiness Index, Lowest Third Citywide',
                    'N count - Postsecondary Enrollment Rate - 6 Months, Lowest Third Citywide',
                    'Metric Value - Postsecondary Enrollment Rate - 6 Months, Lowest Third Citywide',
                    'N count - College and Career Preparatory Course Index, Lowest Third Citywide',
                    'Metric Value - College and Career Preparatory Course Index, Lowest Third Citywide', 'sqrYear']
closing_gap['sqr2019.xlsx'] = closing_gap['sqr2019.xlsx'][closing_gap_cols]
closing_gap['sqr2018.xlsx'] = closing_gap['sqr2018.xlsx'][closing_gap_cols]
closing_gap['sqr2017.xlsx'] = closing_gap['sqr2017.xlsx'][closing_gap_cols]

In [40]:
temp = pd.concat([closing_gap['sqr2019.xlsx'], closing_gap['sqr2018.xlsx'], closing_gap['sqr2017.xlsx']], axis=0)
temp.rename(columns = {'DBN': 'dbn',
                       'N count - 4-Year Graduation Rate - Lowest Third Citywide': 'numGradLowest3rd',
                             'Metric Value - 4-Year Graduation Rate - Lowest Third Citywide': 'pctGradLowest3rd',
                             'N count - 4-Year College Readiness Index, Lowest Third Citywide': 'numCollegeReadyLowest3rd',
                             'Metric Value - 4-Year College Readiness Index, Lowest Third Citywide': 'pctCollegeReadyLowest3rd',
                             'N count - Postsecondary Enrollment Rate - 6 Months, Lowest Third Citywide': 'numPERLowest3rd' ,
                             'Metric Value - Postsecondary Enrollment Rate - 6 Months, Lowest Third Citywide': 'pctPERLowest3rd',
                             'N count - College and Career Preparatory Course Index, Lowest Third Citywide': 'numCCPCILowest3rd',
                             'Metric Value - College and Career Preparatory Course Index, Lowest Third Citywide': 'pctCCPCILowest3rd'}, inplace=True)

In [41]:
closing_gap_cols = ['DBN', 'Metric Value - Graduation Rate, 4-year, lowest third city',
                    'Metric Value - Percentage in Lowest Third City, 4-year College Readiness Index',
                    'Metric Value - Percentage in Lowest Third City, PER, 6 Months After High School',
                    'Metric Value - Percentage in Lowest Third City, College and Career Prep Course Index', 'sqrYear']

closing_gap['sqr2016.xlsx'] = closing_gap['sqr2016.xlsx'][closing_gap_cols]
closing_gap['sqr2016.xlsx'].rename(columns = {'DBN':'dbn',
                                              'Metric Value - Graduation Rate, 4-year, lowest third city':'pctGradLowest3rd',
                    'Metric Value - Percentage in Lowest Third City, 4-year College Readiness Index':'pctCollegeReadyLowest3rd',
                    'Metric Value - Percentage in Lowest Third City, PER, 6 Months After High School':'pctPERLowest3rd',
                    'Metric Value - Percentage in Lowest Third City, College and Career Prep Course Index':'pctCCPCILowest3rd'}, inplace=True)

In [42]:
closing_gap_cols = ['DBN', 'Metric Value - Weighted Diploma Rate, 4 Year - Lowest Third City',
                    'Metric Value - Percentage in Lowest Third City, 4-year Non-Remediation Index',
                    'Metric Value - Percentage in Lowest Third City, PER, 6 Months After High School',
                    'Metric Value - Percentage in Lowest Third City, College and Career Prep Course Index', 'sqrYear']

closing_gap['sqr2015.xlsx'] = closing_gap['sqr2015.xlsx'][closing_gap_cols]
closing_gap['sqr2015.xlsx'].rename(columns = {'DBN':'dbn',
                                              'Metric Value - Weighted Diploma Rate, 4 Year - Lowest Third City':'pctGradLowest3rd',
                    'Metric Value - Percentage in Lowest Third City, 4-year Non-Remediation Index':'pctCollegeReadyLowest3rd',
                    'Metric Value - Percentage in Lowest Third City, PER, 6 Months After High School':'pctPERLowest3rd',
                    'Metric Value - Percentage in Lowest Third City, College and Career Prep Course Index':'pctCCPCILowest3rd'}, inplace=True)

In [43]:
closing_gap_df = pd.concat([closing_gap['sqr2020.xlsx'], temp, closing_gap['sqr2016.xlsx'],closing_gap['sqr2015.xlsx']], axis=0)

In [44]:
closing_gap_df = closing_gap_df[closing_gap_df.dbn.str.len()==6]
closing_gap_df[['numCCPCILowest3rd', 'numCollegeReadyLowest3rd',
       'numGradLowest3rd', 'numPERLowest3rd', 'pctCCPCILowest3rd',
       'pctCollegeReadyLowest3rd', 'pctGradLowest3rd', 'pctPERLowest3rd']] = closing_gap_df[['numCCPCILowest3rd', 'numCollegeReadyLowest3rd',
       'numGradLowest3rd', 'numPERLowest3rd', 'pctCCPCILowest3rd',
       'pctCollegeReadyLowest3rd', 'pctGradLowest3rd', 'pctPERLowest3rd']].apply(pd.to_numeric, errors='coerce')

In [45]:
closing_gap_df.drop(columns=['numCCPCILowest3rd', 'numCollegeReadyLowest3rd'], inplace=True)

In [46]:
closing_gap_df.head()

,dbn,numGradLowest3rd,pctGradLowest3rd,pctCollegeReadyLowest3rd,numPERLowest3rd,pctPERLowest3rd,pctCCPCILowest3rd,sqrYear
3,01M292,6.0,0.833,0.333,19.0,0.579,0.083,2020
4,01M448,31.0,0.903,0.581,20.0,0.500,0.871,2020
5,01M450,23.0,0.739,0.652,11.0,0.636,0.826,2020
6,01M539,13.0,0.769,0.308,1.0,NaN,0.577,2020
7,01M696,2.0,NaN,NaN,1.0,NaN,NaN,2020


In [47]:
add_info['sqr2020.xlsx'].columns = add_info['sqr2020.xlsx'].iloc[1]
add_info['sqr2019.xlsx'].columns = add_info['sqr2019.xlsx'].iloc[1]
add_info['sqr2018.xlsx'].columns = add_info['sqr2018.xlsx'].iloc[0]
add_info['sqr2017.xlsx'].columns = add_info['sqr2017.xlsx'].iloc[0]
add_info['sqr2016.xlsx'].columns = add_info['sqr2016.xlsx'].iloc[0]
add_info['sqr2015.xlsx'].columns = add_info['sqr2015.xlsx'].iloc[0]

In [48]:
add_info_cols = ["DBN", "N Count - Percentage of Students Who Graduated in 4 Years (Out of Students at Level 1 in 8th Grade)", 
                          "Metric Value - Percentage of Students Who Graduated in 4 Years (Out of Students at Level 1 in 8th Grade)",
                          "N Count - Percentage of Students Who Graduated in 4 Years (Out of Students at Level 2 in 8th Grade)",
                          "Metric Value - Percentage of Students Who Graduated in 4 Years (Out of Students at Level 2 in 8th Grade)",
                          "N Count - Percentage of Students Who Graduated in 4 Years (Out of Students at Level 3 or 4 in 8th Grade)",
                          "Metric Value - Percentage of Students Who Graduated in 4 Years (Out of Students at Level 3 or 4 in 8th Grade)"]

add_info['sqr2020.xlsx'] = add_info['sqr2020.xlsx'][add_info_cols]

In [49]:
add_info_cols = ["DBN", "Metric Value - Percentage of Students Who Graduated in 4 Years (Out of Students at Level 1 in 8th Grade)",
                 "N Count - Percentage of Students Who Graduated in 4 Years (Out of Students at Level 1 in 8th Grade)",
                 "Metric Value - Percentage of Students Who Graduated in 4 Years (Out of Students at Level 2 in 8th Grade)",
                 "N Count - Percentage of Students Who Graduated in 4 Years (Out of Students at Level 2 in 8th Grade)", 
                 "Metric Value - Percentage of Students Who Graduated in 4 Years (Out of Students at Level 3 or 4 in 8th Grade)", 
                 "N Count - Percentage of Students Who Graduated in 4 Years (Out of Students at Level 3 or 4 in 8th Grade)"]

add_info['sqr2019.xlsx'] = add_info['sqr2019.xlsx'][add_info_cols]
add_info['sqr2018.xlsx'] = add_info['sqr2018.xlsx'][add_info_cols]

In [50]:
add_info['sqr2020.xlsx']['sqrYear'] = '2020'
add_info['sqr2019.xlsx']['sqrYear'] = '2019'
add_info['sqr2018.xlsx']['sqrYear'] = '2018'

In [51]:
add_info_df = pd.concat([add_info['sqr2020.xlsx'], add_info['sqr2019.xlsx'], add_info['sqr2018.xlsx']], axis=0)

In [52]:
add_info_df.rename(columns={'DBN':'dbn',
       'Metric Value - Percentage of Students Who Graduated in 4 Years (Out of Students at Level 1 in 8th Grade)':'pctGradLevel1',
       'Metric Value - Percentage of Students Who Graduated in 4 Years (Out of Students at Level 2 in 8th Grade)':'pctGradLevel2',
       'Metric Value - Percentage of Students Who Graduated in 4 Years (Out of Students at Level 3 or 4 in 8th Grade)':'pctGradLevel34',
       'N Count - Percentage of Students Who Graduated in 4 Years (Out of Students at Level 1 in 8th Grade)':'numGradLevel1',
       'N Count - Percentage of Students Who Graduated in 4 Years (Out of Students at Level 2 in 8th Grade)':'numGradLevel2',
       'N Count - Percentage of Students Who Graduated in 4 Years (Out of Students at Level 3 or 4 in 8th Grade)':'numGradLevel34'}, inplace=True)

add_info_df = add_info_df[add_info_df.dbn.str.len()==6]

In [53]:
sqr_df = summary_df.merge(stud_achieve_df, how='inner', left_on = ['dbn', 'sqrYear'], right_on = ['dbn', 'sqrYear'])

In [54]:
sqr_df = sqr_df.merge(closing_gap_df, how='inner', left_on = ['dbn', 'sqrYear'], right_on = ['dbn', 'sqrYear'])

In [55]:
sqr_df = sqr_df.merge(add_info_df, how='left', left_on = ['dbn', 'sqrYear'], right_on = ['dbn', 'sqrYear'])

In [56]:
sqr_df.head()

,dbn,sqrEnrollment,econNeedInd,grd8Ela,grd8Math,attdRate,pctChronAbs,sqrYear,numYear1,pct10PlusYear1,numYear1Lowest3rd,pct10PlusYear1Lowest3rd,numYear2,pct10PlusYear2,numYear2Lowest3rd,pct10PlusYear2Lowest3rd,numYear3,pct10PlusYear3,numYear3Lowest3rd,pct10PlusYear3Lowest3rd,numGradLowest3rd,pctGradLowest3rd,pctCollegeReadyLowest3rd,numPERLowest3rd,pctPERLowest3rd,pctCCPCILowest3rd,numGradLevel1,pctGradLevel1,numGradLevel2,pctGradLevel2,numGradLevel34,pctGradLevel34
0,01M292,226.0,0.870,2.78,2.36,0.910,0.360,2020,61.0,0.902,19.0,0.895,64.0,0.906,21.0,0.810,63.0,0.905,21.0,0.762,6.0,0.833,0.333,19.0,0.579,0.083,23,0.826,19,1,7,N<15
1,01M448,492.0,0.842,2.97,3.02,0.913,0.249,2020,117.0,0.983,38.0,0.947,115.0,0.974,35.0,0.943,130.0,0.931,40.0,0.900,31.0,0.903,0.581,20.0,0.500,0.871,50,0.92,76,0.961,71,1
2,01M450,364.0,0.648,3.03,2.96,0.947,0.129,2020,106.0,0.953,31.0,0.935,90.0,0.989,27.0,0.963,77.0,0.935,21.0,0.952,23.0,0.739,0.652,11.0,0.636,0.826,26,0.692,70,0.943,64,0.969
3,01M539,654.0,0.409,3.67,3.89,0.954,0.091,2020,201.0,0.985,64.0,0.969,158.0,0.975,47.0,0.936,135.0,0.970,39.0,0.949,13.0,0.769,0.308,1.0,NaN,0.577,13,N<15,29,1,238,0.987
4,01M696,613.0,0.408,3.85,4.09,0.959,0.084,2020,162.0,0.975,52.0,0.981,165.0,0.988,51.0,0.961,162.0,0.975,48.0,0.938,2.0,NaN,NaN,1.0,NaN,NaN,1,N<15,9,N<15,240,0.992


In [57]:
sqr_df.to_csv('data/clean/sqr.csv',index=False)

#### Location Data

In [58]:
lcgms = pd.read_excel('data/lcgms.xlsx', sheet_name='lcgms')

In [59]:
lcgms['DBN'] = lcgms['ATS System Code'].str.strip()

In [60]:
lcgms.columns

Index(['ATS System Code', 'Location Code', 'Location Name', 'BEDS Number',
       'Managed By Name', 'Location Type Description',
       'Location Category Description', 'Grades', 'Grades Final', 'Open Date',
       'Status Description', 'Building Code', 'Primary Address', 'City',
       'State Code', 'Zip', 'Borough Block Lot', 'Census Tract',
       'Community District', 'Council District', 'NTA', 'NTA_Name',
       'Principal Name', 'Principal Title', 'Principal Phone Number',
       'Fax Number', 'Geographical District Code',
       'Administrative District Code', 'Administrative District Location Code',
       'Administrative District Name', 'Superintendent',
       'Superintendent Title', 'Superintendent Location Code',
       'Community School Sup Name', 'HighSchool Network Location Code',
       'HighSchool Network Name', 'HighSchool Network Superintendent',
       'Executive Superintendent Location code',
       'Executive Superintendent Name', 'Executive Superintendent Title'

In [61]:
lcgms.head()

,ATS System Code,Location Code,Location Name,BEDS Number,Managed By Name,Location Type Description,Location Category Description,Grades,Grades Final,Open Date,Status Description,Building Code,Primary Address,City,State Code,Zip,Borough Block Lot,Census Tract,Community District,Council District,NTA,NTA_Name,Principal Name,Principal Title,Principal Phone Number,Fax Number,Geographical District Code,Administrative District Code,Administrative District Location Code,Administrative District Name,Superintendent,Superintendent Title,Superintendent Location Code,Community School Sup Name,HighSchool Network Location Code,HighSchool Network Name,HighSchool Network Superintendent,Executive Superintendent Location code,Executive Superintendent Name,Executive Superintendent Title,BCO Location Code,BCO Location Name,BCO Executive Director,BCO Executive Director Title,DBN
0,01M015,M015,P.S. 015 Roberto Clemente,3.101000e+11,DOE,General Academic,Elementary,"PK,0K,01,02,03,04,05,SE","PK,0K,01,02,03,04,05",Jul 1 1904,Open,M015,333 EAST 4 STREET,MANHATTAN,NY,10009,1.003740e+09,2601.0,103.0,2.0,MN28,Lower East Side,IRENE SANCHEZ,Principal,212-228-8730,212-477-0931,1,1,M801,COMMUNITY SCHOOL DISTRICT 01,"CHAN, CARRY",CSD Superintendent,M801,"CHAN, CARRY",NaN,NaN,NaN,SS04,"ROSALES, MARISOL",Executive Superintendent,MFSC,Manhattan NYCDOE Borough Office,Steven Aragona,FSC Executive Director,01M015
1,01M019,M019,P.S. 019 Asher Levy,3.101000e+11,DOE,General Academic,Elementary,"PK,0K,01,02,03,04,05,SE","PK,0K,01,02,03,04,05",Jul 1 1956,Open,M019,185 1 AVENUE,MANHATTAN,NY,10003,1.004530e+09,4000.0,103.0,2.0,MN22,East Village,JACQUELINE FLANAGAN,Principal,212-533-5340,212-673-1477,1,1,M801,COMMUNITY SCHOOL DISTRICT 01,"CHAN, CARRY",CSD Superintendent,M801,"CHAN, CARRY",NaN,NaN,NaN,SS04,"ROSALES, MARISOL",Executive Superintendent,MFSC,Manhattan NYCDOE Borough Office,Steven Aragona,FSC Executive Director,01M019
2,01M020,M020,P.S. 020 Anna Silver,3.101000e+11,DOE,General Academic,Elementary,"PK,0K,01,02,03,04,05,SE","PK,0K,01,02,03,04,05",Jul 1 1963,Open,M020,166 ESSEX STREET,MANHATTAN,NY,10002,1.003550e+09,3001.0,103.0,1.0,MN27,Chinatown,SARAH PINTO VIAGRAN,Principal,212-254-9577,212-254-3526,1,1,M801,COMMUNITY SCHOOL DISTRICT 01,"CHAN, CARRY",CSD Superintendent,M801,"CHAN, CARRY",NaN,NaN,NaN,SS04,"ROSALES, MARISOL",Executive Superintendent,MFSC,Manhattan NYCDOE Borough Office,Steven Aragona,FSC Executive Director,01M020
3,01M034,M034,P.S. 034 Franklin D. Roosevelt,3.101000e+11,DOE,General Academic,K-8,"PK,0K,01,02,03,04,05,06,07,08,SE","PK,0K,01,02,03,04,05,06,07,08",Jul 1 1955,Open,M034,730 East 12 Street,Manhattan,NY,10009,1.003810e+09,2800.0,103.0,2.0,MN28,Lower East Side,Bryan Glover,Principal,212-228-4433,212-353-1973,1,1,M801,COMMUNITY SCHOOL DISTRICT 01,"CHAN, CARRY",CSD Superintendent,M801,"CHAN, CARRY",NaN,NaN,NaN,SS04,"ROSALES, MARISOL",Executive Superintendent,MFSC,Manhattan NYCDOE Borough Office,Steven Aragona,FSC Executive Director,01M034
4,01M063,M063,The STAR Academy - P.S.63,3.101000e+11,DOE,General Academic,Elementary,"PK,0K,01,02,03,04,05,SE","PK,0K,01,02,03,04,05",Jul 1 1905,Open,M063,121 EAST 3 STREET,MANHATTAN,NY,10009,1.004310e+09,3200.0,103.0,2.0,MN22,East Village,DARLENE CAMERON,Principal,212-674-3180,212-420-9018,1,1,M801,COMMUNITY SCHOOL DISTRICT 01,"CHAN, CARRY",CSD Superintendent,M801,"CHAN, CARRY",NaN,NaN,NaN,SS04,"ROSALES, MARISOL",Executive Superintendent,MFSC,Manhattan NYCDOE Borough Office,Steven Aragona,FSC Executive Director,01M063


In [62]:
lcgms.groupby(['Administrative District Location Code']).count()

,ATS System Code,Location Code,Location Name,BEDS Number,Managed By Name,Location Type Description,Location Category Description,Grades,Grades Final,Open Date,Status Description,Building Code,Primary Address,City,State Code,Zip,Borough Block Lot,Census Tract,Community District,Council District,NTA,NTA_Name,Principal Name,Principal Title,Principal Phone Number,Fax Number,Geographical District Code,Administrative District Code,Administrative District Name,Superintendent,Superintendent Title,Superintendent Location Code,Community School Sup Name,HighSchool Network Location Code,HighSchool Network Name,HighSchool Network Superintendent,Executive Superintendent Location code,Executive Superintendent Name,Executive Superintendent Title,BCO Location Code,BCO Location Name,BCO Executive Director,BCO Executive Director Title,DBN
Administrative District Location Code,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
DA75,61,61,61,61,61,61,61,61,61,61,61,61,61,61,61,61,61,60,61,61,60,60,61,61,61,61,61,61,61,61,61,61,61,0,0,0,0,0,0,61,61,61,61,61
DA79,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1
HS01,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,0,25,25,25,0,25,25,25,25,25,25,25,25,25,25,25
HS02,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,0,39,39,39,0,39,39,39,39,39,39,39,39,39,39,39
HS03,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,0,41,41,41,0,41,41,41,41,41,41,41,41,41,41,41
HS04,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,0,47,47,47,0,47,47,47,47,47,47,47,47,47,47,47
HS05,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,46,46,47,47,0,47,47,47,0,47,47,47,47,47,47,47,47,47,47,47
HS06,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,0,26,26,26,0,26,26,26,26,26,26,26,26,26,26,26
HS07,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,58,60,60,0,60,60,60,0,60,60,60,60,60,60,60,60,60,60,60


In [63]:
lcgms.drop(columns = ['ATS System Code','Location Code', 'Location Name', 'BEDS Number',
       'Managed By Name', 'Grades', 'Grades Final', 'Open Date',
       'Status Description', 'Building Code', 'Primary Address', 'City',
       'State Code', 
       'Community District', 'Council District', 'NTA', 'NTA_Name',
       'Principal Name', 'Principal Title', 'Principal Phone Number',
       'Fax Number', 
       'Administrative District Code', 'Administrative District Location Code',
       'Administrative District Name', 'Superintendent',
       'Superintendent Title', 'Superintendent Location Code',
       'Community School Sup Name', 'HighSchool Network Location Code',
       'HighSchool Network Name', 'HighSchool Network Superintendent',
       'Executive Superintendent Location code',
       'Executive Superintendent Name', 'Executive Superintendent Title',
       'BCO Location Code', 'BCO Location Name', 'BCO Executive Director',
       'BCO Executive Director Title'], inplace=True)

In [64]:
lcgms.rename(columns={'Location Type Description':'type', 'Location Category Description':'desc',
                      'Zip':'zip', 'Borough Block Lot':'boroughBlockLot', 'Census Tract':'censusTract', 'DBN':'dbn',
                     'Geographical District Code':'geoDis'}, inplace=True)

In [65]:
lcgms.to_csv('data/clean/lcgms.csv',index=False)

#### Demographics Data

In [66]:
demo = pd.read_excel('data/demo.xlsx', sheet_name = 'School')

In [67]:
demo[['sy','sqrYear']] = demo.Year.str.split("-", expand=True)

In [68]:
demo['sqrYear'] = '20'+demo['sqrYear']

In [69]:
demo

,DBN,School Name,Year,Total Enrollment,Grade 3K+PK (Half Day & Full Day),Grade K,Grade 1,Grade 2,Grade 3,Grade 4,Grade 5,Grade 6,Grade 7,Grade 8,Grade 9,Grade 10,Grade 11,Grade 12,# Female,% Female,# Male,% Male,# Asian,% Asian,# Black,% Black,# Hispanic,% Hispanic,# Multiple Race Categories Not Represented,% Multiple Race Categories Not Represented,# White,% White,# Students with Disabilities,% Students with Disabilities,# English Language Learners,% English Language Learners,# Poverty,% Poverty,Economic Need Index,sy,sqrYear
0,01M015,P.S. 015 Roberto Clemente,2015-16,176,14,32,33,39,23,17,18,0,0,0,0,0,0,0,83,0.471591,93,0.528409,9,0.051136,57,0.323864,105,0.596591,3,0.017045,2,0.011364,60,0.340909,16,0.090909,149,0.846591,0.888513,2015,2016
1,01M015,P.S. 015 Roberto Clemente,2016-17,178,17,28,33,27,31,24,18,0,0,0,0,0,0,0,83,0.466292,95,0.533708,14,0.078652,51,0.286517,105,0.589888,4,0.022472,4,0.022472,51,0.286517,12,0.067416,152,0.853933,0.882354,2016,2017
2,01M015,P.S. 015 Roberto Clemente,2017-18,190,17,28,32,33,23,31,26,0,0,0,0,0,0,0,99,0.521053,91,0.478947,20,0.105263,52,0.273684,110,0.578947,2,0.010526,6,0.031579,49,0.257895,8,0.042105,161,0.847368,0.889605,2017,2018
3,01M015,P.S. 015 Roberto Clemente,2018-19,174,13,20,33,30,30,20,28,0,0,0,0,0,0,0,85,0.488506,89,0.511494,24,0.137931,48,0.275862,95,0.545977,1,0.005747,6,0.034483,39,0.224138,8,0.045977,147,0.844828,0.887925,2018,2019
4,01M015,P.S. 015 Roberto Clemente,2019-20,190,14,29,28,38,33,29,19,0,0,0,0,0,0,0,94,0.494737,96,0.505263,27,0.142105,56,0.294737,96,0.505263,2,0.010526,9,0.047368,44,0.231579,17,0.089474,155,0.815789,0.869011,2019,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9091,84X730,Bronx Charter School for the Arts,2015-16,323,0,51,57,55,55,57,48,0,0,0,0,0,0,0,165,0.510836,158,0.489164,2,0.006192,92,0.284830,218,0.674923,6,0.018576,5,0.015480,61,0.188854,49,0.151703,268,0.829721,0.82119,2015,2016
9092,84X730,Bronx Charter School for the Arts,2016-17,320,0,53,53,55,52,53,52,2,0,0,0,0,0,0,168,0.525000,152,0.475000,2,0.006250,76,0.237500,236,0.737500,3,0.009375,3,0.009375,67,0.209375,51,0.159375,235,0.734375,0.840169,2016,2017
9093,84X730,Bronx Charter School for the Arts,2017-18,314,0,50,51,54,51,52,54,2,0,0,0,0,0,0,173,0.550955,141,0.449045,2,0.006369,65,0.207006,243,0.773885,3,0.009554,1,0.003185,68,0.216561,57,0.181529,258,0.821656,0.890586,2017,2018
9094,84X730,Bronx Charter School for the Arts,2018-19,430,0,49,54,49,54,51,53,120,0,0,0,0,0,0,232,0.539535,198,0.460465,2,0.004651,98,0.227907,321,0.746512,6,0.013953,3,0.006977,103,0.239535,71,0.165116,363,0.844186,0.88786,2018,2019


In [70]:
demo.drop(columns = ['School Name', 'Year', 
       'Grade 3K+PK (Half Day & Full Day)', 'Grade K', 'Grade 1', 'Grade 2',
       'Grade 3', 'Grade 4', 'Grade 5', 'Grade 6', 'Grade 7', 'Grade 8',
       '# Multiple Race Categories Not Represented',
       '% Multiple Race Categories Not Represented','sy'], inplace=True)

In [71]:
demo.rename(columns={'DBN':'dbn', 'Total Enrollment':'enrollment',  'Grade 9':'gr9', 'Grade 10':'gr10', 'Grade 11':'gr11',
                     'Grade 12':'gr12', '# Female':'female', '% Female': 'pctFemale', 
                     '# Male':'male', '% Male':'pctMale', '# Asian' : 'asian', '% Asian' : 'pctAsian',
                     '# Black': 'black', '% Black':'pctBlack', '# Hispanic':'hispanic', '% Hispanic':'pctHispanic',
                     '# White':'white', '% White':'pctWhite', '# Students with Disabilities':'swd',
                     '% Students with Disabilities':'pctSwd', '# English Language Learners':'ell', 
                     '% English Language Learners':'pctEll', '# Poverty':'poverty', '% Poverty':'pctPoverty', 
                    'Economic Need Index': 'econInd'}, inplace=True)

In [72]:
demo.loc[demo.econInd == 'Below 5%', 'econInd'] = 0.05
demo.loc[demo.econInd == 'Above 95%', 'econInd'] = 0.95

In [84]:
demo.iloc[:, 1:-1] = demo.iloc[:, 1:-1].apply(pd.to_numeric, errors='coerce')

In [85]:
demo.to_csv('data/clean/demo.csv',index=False)